In [13]:
import sys
import rosbag
import time
import rospy
import os
import argparse
import math
import re
import h5py
import numpy as np
import ros_numpy
import matplotlib.pyplot as plt
import tensorflow as tf
from sensor_msgs.msg import Image, PointCloud2, PointField
import open3d as o3d
from tqdm import tqdm, trange

## user define

In [14]:
folder = '/home/richtong/bag-extract-hdf5/hdf5ifle/'
files = os.listdir(folder)[::-1]
files.sort()

visualize = False

In [15]:
class RunRDPGModel(object):
    def __init__(self):

        model = 'subt_rl495.ckpt-1037'
        output_name = 'Actor/eval/action_out:0'
        input_name = 's:0'
        state_name0 = 'Actor/eval/LSTMCellZeroState/zeros:0'
        state_name1 = 'Actor/eval/LSTMCellZeroState/zeros_1:0'
        out_name0 = 'Actor/eval/rnn/while/Exit_3:0'
        out_name1 = 'Actor/eval/rnn/while/Exit_4:0'
        
        # my_dir = os.path.abspath(os.path.dirname(os.getcwd()))
        my_dir = '/home/richtong/bag-extract-hdf5/'
        input_checkpoint = os.path.join(my_dir, "./model/rdpg/" + model)

        saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=True)
        
        graph = tf.get_default_graph()
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        self.sess = tf.Session(config=config)
        saver.restore(self.sess, input_checkpoint)

        ##########  get tensor ##########
        self.input_s = graph.get_tensor_by_name(input_name)
        self.output_a = graph.get_tensor_by_name(output_name)

        lstm_state0 = graph.get_tensor_by_name(state_name0)
        lstm_state1 = graph.get_tensor_by_name(state_name1)
        self.lstm_state_in = (lstm_state0, lstm_state1)
        
        out_state0 = graph.get_tensor_by_name(out_name0)
        out_state1 = graph.get_tensor_by_name(out_name1)
        self.lstm_state_out = (out_name0, out_name1)

        self.hidden_state = (np.zeros([1,128]),np.zeros([1,128]))
        ##########  get tensor ##########

        self.action_bound ={'linear': 0.35, 'angular': 0.3}

    def scale(self, n, bound):
        return max(min(n, 1), -1)*bound

    def get_action(self, laser):
        laser = laser[np.newaxis, :]  # batch size
        actions, self.hidden_state = self.sess.run([self.output_a, self.lstm_state_out], {
                                        self.input_s: laser, self.lstm_state_in: self.hidden_state})
        action_out = actions[0]

        action_out[0] = self.scale(
            action_out[0], self.action_bound['linear'])
        action_out[1] = self.scale(
            action_out[1], self.action_bound['angular'])
        
        return action_out

    def shutdown(self):
        self.sess.close()


In [16]:
max_dis = 5
min_angle = math.radians(-120)
max_angle = math.radians(120)
angle_incre = math.radians(0.9999)


def pc_to_laser(lidar):
    
    start_a = min_angle
    end_a = start_a + angle_incre
    
    angles = np.zeros(lidar.shape[0])
    for i, p in enumerate(lidar):
        angles[i] = math.atan2(p[1],p[0])

    laser = []
    while start_a < max_angle:
        bundle = lidar[np.where((angles>start_a) & (angles<end_a))][:,:-1]
        if len(bundle)==0:
            d = max_dis
        else:
            bundle = np.linalg.norm(bundle,axis=1)
            d = np.min(bundle)
            d = max_dis if d>max_dis else d
        laser.append(d)
        start_a += angle_incre
        end_a += angle_incre

    laser = np.array(laser)
    return laser


In [17]:
model = RunRDPGModel()

pcd = o3d.geometry.PointCloud()
pcd_l = o3d.geometry.PointCloud()
vis = o3d.visualization.Visualizer()

if visualize:
    vis.create_window()
    vis.add_geometry(pcd)
    
t_files = tqdm(files)

for fname in t_files:
    f = h5py.File(folder+"/"+fname, 'r+')
    f.create_dataset('action', shape=(f['lidar'][:].shape[0],2), maxshape=(None, 2), dtype=np.float16, chunks=True)
        
    for i,lidar in enumerate(f['lidar'][:]) :
        lidar = np.vstack(lidar)
        lidar = np.swapaxes(lidar, 1, 0)
        max_z = np.max(lidar[:,-1])
        min_z = np.min(lidar[:,-1])
        
        lidar = lidar[np.where(lidar[:,-1] > -0.3)]
        lidar = lidar[np.where(lidar[:,-1] < 1)]
        
        laser = pc_to_laser(lidar)
        action = model.get_action(laser)
        f['action'][i] = action
        t_files.set_description('action: %.3f, %.3f'%(action[0],action[1]))
        
        if visualize:
            pcd.points = o3d.utility.Vector3dVector(lidar)
            vis.add_geometry(pcd)
            vis.poll_events()
            vis.update_renderer()
            vis.remove_geometry(pcd)
            vis.remove_geometry(pcd_l)
    
    f.close()
    del f

model.shutdown()
if visualize:
    vis.destroy_window()

AttributeError: 'module' object has no attribute 'import_meta_graph'